In [ ]:
import sys 
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 

print("Numpy version: ", np.__version__)
print("Scipy version: ", sp.__version__)

from non_Hermitian_random_matrix import non_hermitian_symmetry_list, gap_types, is_complexification
from non_Hermitian_random_matrix import DeformHamiltonian, DeformHamiltonianGenerator

from machine_learning import Cluster
from topology_table import DeformTopologicalModel, topology_comparator
from functools import partial

from multiprocessing import Pool
from tools import save_data, save_data_M, load_data, load_data_M

In [ ]:
n_sample = 500
n_band = 8

def worker(non_Hermitian_symmetry_class, n_dim, n_band, gap_type, n_sample):
    print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
        sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

    generator  = DeformHamiltonianGenerator(
            n=n_band, n_dim=n_dim, 
            non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
            verbose=False)
    hamiltonians = generator.generate(n_sample=n_sample)
    models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
    similarity_func = topology_comparator
    cluster = Cluster(similarity_function=similarity_func)
    center_indices, group_number = cluster.fit(models)
    
    filename = "./data/data_{symmetry_class}_{gap_type}_dim_{n_dim}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim)
    save_data(center_indices, group_number, filename)
    
    #return len(group_number)

n_core = 32
for gap_type in gap_types:
    for n_dim in range(9+1):
#        group_number_list = np.zeros(len(non_hermitian_symmetry_list))

#         print("Calculating {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample} in parallel with n_core={n_core}".format(
#             gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample, n_core=n_core))

        _worker = partial(worker, n_dim=n_dim, n_band=n_band, gap_type=gap_type, n_sample=n_sample)
        with Pool(n_core) as pool:
            pool.map(_worker, non_hermitian_symmetry_list)
        
#         filename = "./data/data_table_{gap_type}_dim_{n_dim}.h5".format(gap_type=gap_type, n_dim=n_dim)
#         save_data(non_hermitian_symmetry_list, group_number_list, filename)